In [14]:
import pandas as pd
import numpy as np

In [15]:
closing_price_data = pd.read_pickle("./closing_price_5yr.pkl").dropna(axis = 1)

In [16]:
stock_names = closing_price_data.keys()

In [17]:
closing_price_data.head()

Name,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2013-05-21,31.8140,17.2883,86.3569,57.3325,39.1223,50.7079,33.8299,74.3556,44.02,40.8008,...,28.9535,34.8741,77.7614,41.1599,20.7467,26.7985,45.5818,77.6149,26.3824,31.7795
2013-05-22,31.4670,17.2690,84.9899,57.5534,39.3072,50.5316,33.8299,74.0148,43.17,39.9157,...,28.8902,34.9094,77.2502,40.3427,20.2636,26.6210,45.3545,76.5131,26.0284,32.2701
2013-05-23,31.1267,17.5195,83.2862,57.6559,39.1643,50.6430,33.6147,74.0148,42.77,39.8632,...,28.5650,34.6443,76.9151,40.3816,20.5167,26.1915,45.0103,76.5993,26.0379,31.5872
2013-05-24,31.0246,17.5387,82.6027,58.0484,38.1976,50.5131,33.8658,73.7457,42.43,39.7580,...,28.4566,34.8122,76.6972,40.0314,20.5167,26.0048,44.9518,76.5227,26.3059,32.1835
2013-05-28,31.5963,17.4520,82.8404,57.5645,38.1556,50.4481,34.5833,74.5260,42.91,39.8544,...,28.7096,35.3070,77.4094,40.9653,20.7237,26.2009,45.4844,77.3945,26.8800,32.9530


In [18]:
ticket = ['MMM','AXP','AAPL','BA','CAT','CVX','CSCO','KO','DIS','D','XOM','GE','GS','HD','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PFE','PG','TRV','UTX','UNH','VZ','V','WMT']
stock_data = closing_price_data[ticket]
stock_data.shape

(1259, 30)

# Now is for the partitions of data

In [19]:
class my_mat_funcs():
    import numpy as np
    def mat_mults(*args):
        if(len(args)==1):
            return args[0]
        else:
            return np.matmul(args[0],mat_mults(*args[1:]))

In [20]:
from collections import OrderedDict
########## Postcondition: calculate each w ##################
def optimal_solution(stock_data_partition, sigma = 'sample', inverse = False):
    
    logret = np.log(stock_data_partition).diff()
    mu = logret[1:].mean()
    if(inverse):
        s, _ = sigma.shape
        one = np.ones((s,1), dtype = float)
        w = 1 / mat_mults(np.transpose(one), sigma, one)[0] * mat_mults(sigma, one)
    else:
        import cvxpy as cvx
        if(sigma == 'sample'):
            sigma = logret.cov()
        s, _ = sigma.shape
        w = cvx.Variable(s)# the length of w is x
        risk = cvx.quad_form(w, sigma.as_matrix())
        prob = cvx.Problem(cvx.Minimize(risk),
                           [cvx.sum_entries(w) == 1])
        prob.solve()
    
    return w.value
    

In [21]:
#### Postcondition: calculate all w ##############
def Optimization_At_Each_Period(stock_data,days_of_period):
    w=[]
    for current_period_num in range(int(np.floor( (len(stock_data)/days_of_period) ))):
        stock_data_partition = stock_data[current_period_num*days_of_period:(current_period_num+1)*days_of_period].copy()
        w.append(optimal_solution(stock_data_partition))
    return np.column_stack(w)
#     if len(stock_data)/days_of_period-int(len(stock_data)/days_of_period)==0:
#         return np.column_stack(w)
#     else:
#         print ("Last Trading Period Not Finished")
#         last_start=int(np.floor( (len(stock_data)/days_of_period) ))*days_of_period
#         last_end=len(stock_data)-1
#         w.append(optimal_solution(stock_data[last_start:last_end]))
#         return np.column_stack(w)

In [22]:
#### Postcondition: list of return chunks ################
def return_over_time(stock_data,days_of_period):
    re=[]
    for current_period_num in range(int(np.floor( (len(stock_data)/days_of_period) ))):
        before = stock_data[(days_of_period*current_period_num+1):(days_of_period*(current_period_num+1)+1)]
        after = stock_data[(days_of_period*current_period_num):(days_of_period*(current_period_num+1))]
        tmp = np.log(np.array(after)/np.array(before))
        tmp = pd.DataFrame(tmp,columns = after.columns)
        re.append(tmp)
        
    if len(stock_data)/days_of_period-int(len(stock_data)/days_of_period)==0:
        return re
    else:
        print ("Last Trading Period Not Finished")
        last_start=int(np.floor( (len(stock_data)/days_of_period) ))*days_of_period
        last_end=len(stock_data)-1
        before = stock_data[(last_start-1):(last_end-1)]
        after = stock_data[last_start:last_end]
        tmp = np.log(np.array(after)/np.array(before))
        tmp = pd.DataFrame(tmp,columns = after.columns)
        re.append(tmp)
        return re

In [23]:
days_of_period = 252

In [24]:
def Realized_Return(stock_data,days_of_period, start = '2014-05-21'):
    n = days_of_period
    es = pd.Series(np.arange(n, ))
    Mat_Optimization_At_Each_Period = Optimization_At_Each_Period(stock_data,days_of_period)  # w
    Mat_return_over_time = return_over_time(stock_data,days_of_period)  # return dataframe by chunks
    Total_time = Mat_Optimization_At_Each_Period.shape[1]  # Number of periods
    ss = shortsize(Mat_Optimization_At_Each_Period)  # Size of short side
    daily_r = pd.DataFrame()
    for ticker in range(Total_time):
        tmp = pd.DataFrame(np.matmul(Mat_return_over_time[ticker + 1], Mat_Optimization_At_Each_Period[:,ticker]))
        daily_r  = pd.concat([daily_r, tmp])
    return average_return
    

In [25]:
np.arange(0,20,6)

array([ 0,  6, 12, 18])

In [26]:
shortsize(Mat_Optimization_At_Each_Period)

NameError: name 'shortsize' is not defined

In [27]:
def shortsize(Mat_Optimization_At_Each_Period):
    ss = np.empty(Mat_Optimization_At_Each_Period.shape[1])  # take number of columns (number of periods)
    for i in range(Mat_Optimization_At_Each_Period.shape[1]):
        absw = np.array([np.min(np.array([np.asscalar(j),0])) for j in Mat_Optimization_At_Each_Period[:,i]])
        ss[i] = np.sum(np.abs(absw)) / np.sum(np.abs(Mat_Optimization_At_Each_Period[:,i]))
    return ss

In [32]:
def TurnOver(k,stock_data,days_of_period):
    if k==1:
        Wkm1=np.matrix(np.zeros((len(stock_data.keys()),1)))
    Wk = Optimization_At_Each_Period(stock_data,days_of_period)[:,k-1]
    Wkm1 = Optimization_At_Each_Period(stock_data,days_of_period)[:,k-2]
    ROT=return_over_time(stock_data,days_of_period)
    RPlus1 = np.ones(np.matrix(ROT[k]).shape)+np.matrix(ROT[k])
    R1row = np.prod(RPlus1)
    return Wk-R1row*Wkm1

In [33]:
TurnOver(3,stock_data,252)

Last Trading Period Not Finished


matrix([[ 0.05451399],
        [ 0.17116212],
        [ 0.04548382],
        [-0.09574047],
        [ 0.05579512],
        [-0.09302996],
        [-0.06597448],
        [ 0.28591557],
        [ 0.06356186],
        [ 0.20115559],
        [ 0.05377596],
        [-0.07766061],
        [-0.00994574],
        [ 0.02294318],
        [ 0.02074876],
        [-0.00592271],
        [ 0.24289829],
        [-0.05651342],
        [ 0.1129356 ],
        [-0.09614955],
        [-0.07210555],
        [ 0.01933933],
        [ 0.09799251],
        [ 0.09379296],
        [-0.08819687],
        [ 0.01948447],
        [ 0.02632329],
        [ 0.0783836 ],
        [-0.05386637],
        [ 0.04502002]])

In [28]:
daily_r = pd.DataFrame()
for ticker in range(Total_time):
    daily_r  = pd.concat([daily_r, pd.DataFrame(np.matmul(Mat_return_over_time[ticker + 1], Mat_Optimization_At_Each_Period[:,ticker]))])

NameError: name 'Total_time' is not defined

In [ ]:
stock_data.shape

In [ ]:
np.matmul(A,B)

In [ ]:
Mat_return_over_time[ticker].iloc[:,466]

In [ ]:
Mat_Optimization_At_Each_Period.shape

In [ ]:
len(Mat_return_over_time)

In [ ]:
Mat_return_over_time[0].shape

In [ ]:
Mat_Optimization_At_Each_Period.shape

In [ ]:
Mat_return_over_time[0]

In [ ]:
closing_price_data.shape

In [ ]:
ROT=return_over_time(closing_price_data,21)

In [ ]:
ROT.shape

In [ ]:
OMT= Optimization_At_Each_Period(closing_price_data,21)

In [ ]:
Realized_Return(closing_price_data,21)

In [ ]:
OMT.shape

In [ ]:
RR1.shape[1]

In [ ]:
RR1[0,:]

In [ ]:
np.matrix(np.cov(closing_price_data[0:21]))

In [ ]:
np.matrix(np.ones(21))

In [ ]:
np.linalg.inv(np.matmul(
    np.matmul(np.matrix(np.ones(21)),
              np.matrix(np.cov(closing_price_data[0:21]))),
    np.transpose(np.matrix(np.ones(21)))))

In [ ]:
np.linalg.inv(np.matrix(np.ones(21)))

In [29]:
import cvxpy as cvx

s, _ = sigma.shape

w = cvx.Variable(s)# the length of w is x


risk = cvx.quad_form(w, sigma.as_matrix())
prob = cvx.Problem(cvx.Minimize(risk), 
               [cvx.sum_entries(w) == 1])
prob.solve()
w.value

NameError: name 'sigma' is not defined

In [ ]:
closing_price_data[0:1]

In [ ]:
closing_price_data[21:22]

In [ ]:
np.log(np.matrix(closing_price_data[21:22])/np.matrix(closing_price_data[0:1]))

In [ ]:
np.matrix(closing_price_data[21:22])